# MGRS project

##### TODO

- Try to obtain similar results as Sofia Antipolis
- Rebuild the analysis from fastq

## 1) Introduction

Laboratory has developped cellular and animal tools to have a better understanding of intrinsic (plasmocyte) and extrinsic (kidney toxicity) impact of some immunoglobulins (Ig).
Part 1 : Some anormal Igs (truncates or incompletes) are toxic for plasmocytes producing it. Factors of this toxicity will be study comparing plasmocytes transcriptome secreting normal vs anormal Ig.
For each condition, 3 duplicates.
Part 2 : We have shown that some Igs could remotly induce kidney disrupt and we want to analyse mechanisms leading to this toxicity. This second part is the 3rd biological replicate of an analysis already start with ReaIg 1922 project and ToxIg2. Analysis done on RNApolyA

## 2) Data

### 2.1) Raw reads

Fastq sent from Sofia-Antipolis

In [ ]:
### CODE ###
# Untar file
tar -xvf sirac.tar;

### 2.2) Mus Musculus

#### 2.2.1) Reference genome & associated GFF

Downloaded from the "Genome Reference Consortium", latest realease :

In [ ]:
### CODE ###
url='ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/GCA_000001635.8_GRCm38.p6';
axel -q $url/GCA_000001635.8_GRCm38.p6_genomic.fna.gz;
axel -q $url/GCA_000001635.8_GRCm38.p6_genomic.gff.gz;
unpigz *_GRCm38.p6_genomic.fna.gz;
unpigz *_GRCm38.p6_genomic.gff.gz;

#### 2.2.2) Reference transcriptome

###### Same question than PTCB :

Which refererence transcriptome to use with Kallisto ? 

Asked question : I plan to work with Kallisto on the mus musculus transcriptome : Which DB should I use to retrieve the transcriptome ? : Genbank, Ensembl or UCSC ? 

Answer (Kallisto authors) : "Anyway, our links would simply point you to the ensembl transcriptome when it exists. We typically use this as it is fairly complete (by comparison with others) and the transcript to gene name mapping is pretty nice (also by comparison with others)." 

Thus we retrieve transcriptome from Ensembl latest release and use cdna + ncrna (to cover almost as many genes as the GENCODE annotation used with featureCounts).

######  Ensembl release-91

In [ ]:
### CODE ###
url='ftp://ftp.ensembl.org/pub/current_fasta/mus_musculus';
axel -q $url/cdna/Mus_musculus.GRCm38.cdna.all.fa.gz;
axel -q $url/ncrna/Mus_musculus.GRCm38.ncrna.fa.gz;
cat Mus_musculus.GRCm38.cdna.all.fa.gz \
    Mus_musculus.GRCm38.ncrna.fa.gz \
    > mm_ensembl_grcm38_r91_cdna_ncrna.fasta.gz;
rm Mus_musculus.GRCm38.cdna.all.fa.gz Mus_musculus.GRCm38.ncrna.fa.gz;
gunzip mm_ensembl_grcm38_r91_cdna_ncrna.fasta.gz;

#### 2.2.3) GTF annotations

For future use with featureCounts.

Downloaded from "GENCODE". 

"It contains the comprehensive gene annotation on the reference chromosomes, scaffolds, assembly patches and alternate loci (haplotypes)". 

More precisely, it annotate a lot of gene/transcript biotypes : https://www.gencodegenes.org/gencode_biotypes.html.

In [3]:
### CODE ###
url='ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M16';
axel -q $url/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf.gz;
unpigz *_scaff.annotation.gtf.gz;

###### Remove reference genome sequences if they are not in the annotation used for featureCounts

In [ ]:
### CODE ###
python rename_fna_for_gtf_annotation.py -f ../../Data/MGRS/GCA_000001635.8_GRCm38.p6_genomic.fna -g ../../Data/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf

Content of rename_fna_for_gtf_annotation.py :

In [ ]:
### CODE ###
import os
import sys
import getopt
import os.path
import shutil

### A script allowing change header of fna (.fasta) file with gtf annotation


def usage():
	print('Usage:')
	print('\tpython '+sys.argv[0]+' -d <directory_of_fasta_files>')
	print('\t\t-f or --fna : fna file')
	print('\t\t-g or --gtf : gtf file')

def main(argv):

	fna_file = ""
	gtf_file = ""
		
	try:
		opts, args = getopt.getopt(sys.argv[1:], 'f:g:', ['fna=', 'gtf=', 'help'])
	except getopt.GetoptError:
		usage()
		sys.exit(2)

	if not opts :
		usage()
		sys.exit(2)
	for opt, arg in opts:
		if opt in ('-h', '--help'):
			usage()
			sys.exit(2)
		elif opt in ('-f', '--fna'):
			fna_file = arg
		elif opt in ('-g', '--gtf'):
			gtf_file = arg
		else:
			usage()
			sys.exit(2)

	if fna_file[-4:]!=".fna" or os.path.exists(fna_file)==False:
		print("The fna file is missing or not .fna !\n")
		usage()
		sys.exit(2)
	if gtf_file[-4:]!=".gtf" or os.path.exists(gtf_file)==False:
		print("The gtf file is missing or not .gtf !\n")
		usage()
		sys.exit(2)
	
	output_rename = fna_file[:-4]+"_renamed"+".fna"
	output_subsample = fna_file[:-4]+"_subsampled"+".fna"

	print('\n-----------------------------------------')
	print('Fna file : '+fna_file)
	print('GTF file : '+fna_file)
	print('Output file rename: '+output_rename)
	print('Output file rename: '+output_subsample)
	print('-----------------------------------------\n')

	list_header_fna=[]
	list_header_gtf=[]
	shared_header=[]
	f_output_rename = open(output_rename, 'a')

	dict_names={"CM000994.2":"chr1", "CM000995.2":"chr2", "CM000996.2":"chr3", "CM000997.2":"chr4", "CM000998.2":"chr5", "CM000999.2":"chr6", "CM001000.2":"chr7", "CM001001.2":"chr8", "CM001002.2":"chr9", "CM001003.2":"chr10", "CM001004.2":"chr11", "CM001005.2":"chr12", "CM001006.2":"chr13", "CM001007.2":"chr14", "CM001008.2":"chr15", "CM001009.2":"chr16", "CM001010.2":"chr17", "CM001011.2":"chr18", "CM001012.2":"chr19", "CM001013.2":"chrX", "CM001014.2":"chrY", "AY172335.1":"chrM", "JH792831.2":"JH792831.1", "KQ030493.2":"KQ030493.1"}
	for record in SeqIO.parse(fna_file, 'fasta'):
		if record.id in dict_names:
			record.id = dict_names[record.id]
			f_output_rename.write(">"+record.id+"\n")
			f_output_rename.write(str(record.seq)+"\n")
		list_header_fna.append(record.id)

	f_gtf = open(gtf_file, "r")
	for line in f_gtf:
		if not line.startswith('##'):
			if line.split('\t')[0] not in list_header_gtf:
				list_header_gtf.append(line.split('\t')[0])

	count_unique_fna=0
	count_unique_gtf=0
	count_both=0

	for i in list_header_fna:
		if i in list_header_gtf:
			shared_header.append(i)
			count_both+=1
		else:
			count_unique_fna+=1

	for i in list_header_gtf:
		if i not in list_header_fna:
			count_unique_gtf+=1

	print("Number of unique in fna : "+str(count_unique_fna))
	print("Number of unique in gtf : "+str(count_unique_gtf))
	print("Number of shared : "+str(count_both))

	f_output_subsample = open(output_subsample, 'a')

	for record in SeqIO.parse(output_rename, 'fasta'):
		if record.id in shared_header:
			f_output_subsample.write(">"+record.id+"\n")
			f_output_subsample.write(str(record.seq)+"\n")

if __name__ =='__main__':
	main(sys.argv[1:])

#### 2.2.4) Ensembl vs GENCODE

We want at some point to compare results obtained with the "genomic path", i.e. STAR + featureCounts + DESeq2 vs results from the "transcriptomic path", i.e. Kallisto + DESeq2. Thus, we need make sure that featureCounts (using GENCODE annotations) and Kallisto (using transcriptomes sequences) will compute reads counts at the gene level for approximately the same genes. 

In [1]:
### CODE ###
python ensembl_vs_gencode.py -e ../../Data/MGRS/mm_ensembl_grcm38_r91_cdna_ncrna.fasta -g ../../Data/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf

Content of ensembl_vs_gencode.py :

In [ ]:
### CODE ###
import os
import sys
import getopt
import os.path
import shutil
from Bio import SeqIO 

### A script allowing find shared annotation gencode/ensembl


def usage():
	print('Usage:')
	print('\tpython '+sys.argv[0]+' -e <ensembl file> -g <gencode file>')
	print('\t\t-e or --ensembl : fasta file')
	print('\t\t-g or --gencode : gtf file')

def main(argv):

	ensembl_file = ""
	gencode_file = ""
		
	try:
		opts, args = getopt.getopt(sys.argv[1:], 'e:g:', ['ensembl=', 'gencode=', 'help'])
	except getopt.GetoptError:
		usage()
		sys.exit(2)

	if not opts :
		usage()
		sys.exit(2)
	for opt, arg in opts:
		if opt in ('-h', '--help'):
			usage()
			sys.exit(2)
		elif opt in ('-e', '--ensembl'):
			ensembl_file = arg
		elif opt in ('-g', '--gencode'):
			gencode_file = arg
		else:
			usage()
			sys.exit(2)

	if ensembl_file[-6:]!=".fasta" or os.path.exists(ensembl_file)==False:
		print("The ensembl file is missing or not .fasta !\n")
		usage()
		sys.exit(2)
	if gencode_file[-4:]!=".gtf" or os.path.exists(gencode_file)==False:
		print("The gencode file is missing or not .gtf !\n")
		usage()
		sys.exit(2)

	print('\n-----------------------------------------')
	print('Ensembl file : '+ensembl_file)
	print('Gencode file : '+gencode_file)
	print('-----------------------------------------\n')

	annot_gencode=[]
	annot_ensembl=[]
	shared_annot=[]

	f_gencode_file = open(gencode_file, "r")
	for line in f_gencode_file:
		gene_name=""
		if not line.startswith('##'):
			gene_name=line.split('\t')[8].split(";")[0].split("\"")[1].split(".")[0]
			#print(gene_name)
			if gene_name not in annot_gencode:
				annot_gencode.append(gene_name)

	for record in SeqIO.parse(ensembl_file, 'fasta'):
		record.description = record.description.split(" ")[3].split(":")[1].split(".")[0]
		if record.description not in annot_ensembl:
			annot_ensembl.append(record.description)

	count_unique_ensembl=0
	count_unique_gencode=0
	count_both=0

	for i in annot_gencode:
		if i in annot_ensembl:
			shared_annot.append(i)
			count_both+=1
		else:
			count_unique_gencode+=1

	for i in annot_ensembl:
		if i not in annot_gencode:
			count_unique_ensembl+=1

	print("Number of unique in gencode : "+str(count_unique_gencode))
	print("Number of unique in ensembl : "+str(count_unique_ensembl))
	print("Number of shared : "+str(count_both))

if __name__ =='__main__':
	main(sys.argv[1:])

Number of unique in gencode : 3122

Number of unique in ensembl : 19

Number of shared : 50805

50805 genes are common (94.87%), so we consider that we can compare DE results with counts coming from featureCounts and from Kallisto.

## 3) Raw reads QC & filtering

### 3.1) FastQC

In [ ]:
### CODE ###
fastqc /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2016.runs/161202_sirac_wtr_1003/*
fastqc /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/170411_Sirac_wtr_1024/170411_Sirac_wtr_1024_mouse/*
fastqc /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/171018_sirac_wtr_1069/*
mkdir /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2016.runs/161202_sirac_wtr_1003/*.zip /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2016.runs/161202_sirac_wtr_1003/*.html /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/170411_Sirac_wtr_1024/170411_Sirac_wtr_1024_mouse/*.zip /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/170411_Sirac_wtr_1024/170411_Sirac_wtr_1024_mouse/*.html /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/171018_sirac_wtr_1069/*.zip /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/171018_sirac_wtr_1069/*.html /media/sf_raid/Data/MGRS/FastQC

### 3.2) MultiQC

We compile FastqQC analysis for all the data, to highlight potential run effect.

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/FastQC
multiqc -o 161202 *S7*.zip *S8*.zip *S9*.zip
multiqc -o 170411 *S10*.zip *S11*.zip *S12*.zip
multiqc -o 171018 *S13*.zip *S14*.zip *S15*.zip

Visualize QC results :

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/FastQC
chromium-browser 161202/multiqc_report.html;
chromium-browser 170411/multiqc_report.html;
chromium-browser 171018/multiqc_report.html;

## 4) Assign contigs based on subject taxonomy

For this purpose, we are using blast+_2.7.1

##### Get taxdb for blast+ locally (useful to directly retrieve staxids when database = nt or nr) :

In [ ]:
#### CODE ###
wget ftp://ftp.ncbi.nlm.nih.gov/blast/db/taxdb.tar.gz;
wget ftp://ftp.ncbi.nlm.nih.gov/blast/db/taxdb.tar.gz.md5;
for f in *.md5;
do tmp=$(md5sum -c $f);
   if [[ $tmp == *": OK" ]];
   then tar -zxvf  ${f%.*};
        rm $f ${f%.*};
   fi;
done;

Ps : Don't forget to move tax files in the same dir as your other databases (nt, nr, etc...).<br>
And don't forget to add the BLASTDB variable to your .bashrc (export BLASTDB = 'your_path).

### 4.1) Blastn vs nt

The goal is to blast assembled contigs vs nt and afterwards check if contigs matched vs a plant sequence, a non-plant sequence or nothing. It's one of the first step to "filter" the raw/draft transcriptome assembly.

In [ ]:
#### CODE ###
biscem='/home/erwann/Desktop/RTDG/BISCEm';
cd $biscem/Data;
blast='/home/erwann/Software/Blast_2.7.1/bin';
db='/home/erwann/Software/ncbi-blast-2.5.0+/blastdb';
for id in 'hess';
do unpigz $id.fasta.gz;
   $blast/blastn -query $id.fasta \
                 -db $db/nt \
                 -out $biscem/Output/$id'_vs_nt.tsv' \
                 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send \
                          evalue bitscore qlen slen saccver staxids sskingdoms sblastnames stitle" \
                 -num_threads 8 \
                 -culling_limit 1;
   pigz $id.fasta;
done;

### 4.2) Parse blastn result

Nb : The idea on how to check if a contig matched to plant with the staxids field came from this post : https://www.biostars.org/p/163595/#163603. <br>

What it does basically is :<br>
"staxids" allows us to query the NCBI taxonomy database for the lineage of a taxon information with the tool eutils.<br>
We can then parse this eutils xml result to check if kingdom <=> "Viridiplantae".

Below, a dummy example for staxids = 3357 (Pseudotsuga menziesii <=> Douglas) :

In [1]:
#### CODE ###
# Retrive "RECOFGE" + superkingdom, in this order : E_SK_R_E_C_O_F_G
efetch -db taxonomy \
       -id 3357 \
       -format xml \
       | xtract -pattern Taxon \
                  -sep '@' \
                  -element TaxId,ScientificName \
                -division LineageEx \
                -group Taxon \
                  -if Rank -equals superkingdom \
                  -or Rank -equals kingdom \
                  -or Rank -equals phylum \
                  -or Rank -equals class \
                  -or Rank -equals order \
                  -or Rank -equals family \
                  -or Rank -equals genus \
                    -sep '@' \
                    -element Rank,ScientificName;

3357@Pseudotsuga menziesii	superkingdom@Eukaryota	kingdom@Viridiplantae	phylum@Streptophyta	order@Pinales	family@Pinaceae	genus@Pseudotsuga


##### sort_plant_hit_vs_nt.py

In [ ]:
#### CODE ###
#!/usr/bin/env python
import os
import sys
import timeit

usage = '\t --------\n' \
        '\t| usage  : python sort_plant_hit_vs_nt.py f1 f2\n' \
        '\t| input  : f1 = blastn.tsv (vs nt)\n' \
        '\t| input  : f2 = seqs.fasta (blastn queries)\n' \
        '\t| output : "f2"_1.fasta (plant_hit)\n' \
        '\t| output : "f2"_2.fasta (non_plant_hit)\n' \
        '\t| output : "f2"_3.fasta (no_hit)\n' \
        '\t --------'

if len(sys.argv) != 3:
    print(usage)
    sys.exit()

##############
### Step 1 ###
##############
print('\n\tStep 1) Retrieve taxonomic infos for each staxids with efetch')
t0 = timeit.default_timer()

# For each line in TSV (f1), fill staxids_set
staxids_set = set()
with open(sys.argv[1], 'r') as tsv:
    for row in tsv:
        columns = row.split('\t')
        # Sometimes you have more than one staxids for an entry
        staxids = columns[15].split(';')
        for i in staxids:
            staxids_set.add(i)

# Use staxids_set as query with efetch & store result
# Don't give more than let's say 500 entries at a time to avoid timeout
staxids_li = list(staxids_set)
staxids_sub_li = [staxids_li[x:x + 500]
                  for x in range(0, len(staxids_li), 500)]
efetch_li = []
for item in staxids_sub_li:
    staxids_input = ','.join(str(z) for z in item)
    # Details about "cmd" : https://www.biostars.org/p/163595/#271497
    cmd = ('efetch -db taxonomy -id ' + staxids_input + ' -format xml | xtract '
           '-pattern Taxon -sep \'@\' -element TaxId,ScientificName -division '
           'LineageEx -group Taxon -if Rank -equals superkingdom -or Rank '
           '-equals kingdom -or Rank -equals phylum -or Rank -equals class'
           ' -or Rank -equals order -or Rank -equals family -or Rank -equals'
           ' genus -sep \'@\' -element Rank,ScientificName')
    cmd_result = os.popen(cmd).read()
    cmd_result_split = cmd_result.split('\n')
    for i in cmd_result_split:
        efetch_li.append(i)

# Create a dict associating key=staxid with value=list=tax_infos
taxonomy_dic = {}
for line in efetch_li:
    field = line.split('\t')
    tax_ids = field[0].split('@')
    # Sometimes more than one staxid is associated to an entry
    # e.g. "170850@3666@Cucurbita hybrid cultivar"
    for i in tax_ids[:-1]:
        taxonomy_dic.setdefault(i, [None, None, None, None, None, None, None])
        for item in field:
            if 'superkingdom@' in item:
                taxonomy_dic[i][0] = item.split('@')[-1]
            elif 'kingdom@' in item:
                taxonomy_dic[i][1] = item.split('@')[-1]
            elif 'phylum@' in item:
                taxonomy_dic[i][2] = item.split('@')[-1]
            elif 'class@' in item:
                taxonomy_dic[i][3] = item.split('@')[-1]
            elif 'order@' in item:
                taxonomy_dic[i][4] = item.split('@')[-1]
            elif 'family@' in item:
                taxonomy_dic[i][5] = item.split('@')[-1]
            elif 'genus@' in item:
                taxonomy_dic[i][6] = item.split('@')[-1]

print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

##############
### Step 2 ###
##############
print('\tStep 2) Assign contigs best hit to plant or non-plant')
t0 = timeit.default_timer()

# Assign contigs best hits to plant or non-plant based on taxonomy_dic infos
qseqid_set, viridi_hit_set, non_viridi_hit_set = set(), set(), set()
with open(sys.argv[1], 'r') as tsv:
    for row in tsv:
        columns = row.split('\t')
        qseqid, staxids = columns[0], columns[15].split(';')[0]
        # Check if we encounter qseqid for the first time <=> best hit
        if not qseqid in qseqid_set:
            if taxonomy_dic[staxids][1] == 'Viridiplantae':
                viridi_hit_set.add(qseqid)
            else:
                non_viridi_hit_set.add(qseqid)
        qseqid_set.add(qseqid)

print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

##############
### Step 3 ###
##############
print('\tStep 3) Find contigs with no hits')
t0 = timeit.default_timer()

# Read initial FASTA (f2) & check intersection with viridi_hit_set & non_viridi_hit_set
# We can deduce contigs with no hit from this intersection
cpt = 0
no_hit_set = set()
with open(sys.argv[2], 'r') as fa:
    for line in fa:
        if line.startswith('>'):
            cpt += 1
            line = line.lstrip('>')
            fields = line.split()
            no_hit_set.add(fields[0])

before_union = len(no_hit_set)
no_hit_set = no_hit_set - viridi_hit_set
no_hit_set = no_hit_set - non_viridi_hit_set

print('\t\t- number of seqs (>) in FASTA : ' + str(cpt))
print('\t\t- number of headers added to initial set is ' + str(before_union))
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

##############
### Step 4 ###
##############
print('\tStep 4) Create output files')
t0 = timeit.default_timer()

# Create input files (sequence IDs list) for seqtk
file_2 = sys.argv[2].split('/')
sample = file_2[-1].split('.')[0]
with open(sample + '_plant_hit.temp', 'w') as out:
    for item in viridi_hit_set:
        out.write(item + "\n")
with open(sample + '_non_plant_hit.temp', 'w') as out:
    for item in non_viridi_hit_set:
        out.write(item + "\n")
with open(sample + '_no_hit.temp', 'w') as out:
    for item in no_hit_set:
        out.write(item + "\n")

# Create output files (plant_hit = 1, non-plant_hit = 2, no-hit = 3) with seqtk
os.system('seqtk subseq ' + sys.argv[2] + ' ' + sample +
          '_plant_hit.temp > ' + sample + '_1.fasta')
os.system('seqtk subseq ' + sys.argv[2] + ' ' + sample +
          '_non_plant_hit.temp > ' + sample + '_2.fasta')
os.system('seqtk subseq ' + sys.argv[2] + ' ' + sample +
          '_no_hit.temp > ' + sample + '_3.fasta')

sum_seqs = int(len(viridi_hit_set)) + int(len(non_viridi_hit_set)) + int(len(no_hit_set))
print('\t\t- number of seqs with plant_hit : ' + str(len(viridi_hit_set)))
print('\t\t- number of seqs in non_plant_hit : ' + str(len(non_viridi_hit_set)))
print('\t\t- number of seqs with no_hit : ' + str(len(no_hit_set)))
print('\t\t- sum of the 3 above : ' + str(sum_seqs))
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

os.system('rm *.temp')

What it does : <br>
Take the blastn TVS file and the contig FASTA file as input.<br>
Then check for each contig best hit if kingdom <=> "Viridiplantae".<br>
If so, we consider the contig to have a plant hit (output file "_1").<br>
Else, we consider the contig to have a non-plant hit (output file "_2").<br>
Then, iterate other the contig file to collect contigs name & check intersection with contigs having plant / non-plant hits. This allow to determine contigs with no hits (output file "_3").

Launch the script :

In [3]:
#### CODE ###
biscem='/home/erwann/Desktop/RTDG/BISCEm';
cd $biscem/Output;
for id in 'trinity_cdhitest';
do unpigz $biscem/Data/$id'_renamed.fasta.gz';
   python $biscem/Git/sort_plant_hit_vs_nt.py $id'_renamed_vs_nt.tsv' $biscem/Data/$id'_renamed.fasta';
   pigz $biscem/Data/$id'_renamed.fasta';
done;


	Step 1) Retrieve taxonomic infos for each staxids with efetch
		=> 35.252 seconds

	Step 2) Assign contigs best hit to plant or non-plant
		=> 0.534 seconds

	Step 3) Find contigs with no hits
		- number of seqs (>) in FASTA : 799102
		- number of headers added to initial set is 799102
		=> 3.132 seconds

	Step 4) Create output files
		- number of seqs with plant_hit : 98818
		- number of seqs in non_plant_hit : 92665
		- number of seqs with no_hit : 607619
		- sum of the 3 above : 799102
		=> 6.974 seconds



The above was done for each known reference transcriptome + UGMA assemblies. Then basic stats were computed on output FASTA files :

In [ ]:
#### CODE ###
for f in  hess*.fasta;
do echo $f; perl ../Script/assemblyStats.pl $f;
done;

Results are compiled here : https://docs.google.com/spreadsheets/d/1hYWprws5gd2-W2vgMux2lVAtXx5TVm4IenuwZr6DcF4/edit#gid=0

### 4.3) Blastx vs nr | plant_refseq

This step is especially useful for contigs which have previously failed to be assigned to "Viridiplantae" vs nt. <br>
Blastx vs a protein subject database may highlight more informative contigs. <br>

#### 4.3.1) Create DB

In [ ]:
### CODE ###
biscem='/home/erwann/Desktop/RTDG/BISCEm';
blast='/home/erwann/Software/Blast_2.7.1/bin';
unpigz $biscem/Data/plant_ref_seq.faa.gz;
$blast/makeblastdb -in $biscem/Data/plant_ref_seq.faa -input_type fasta -dbtype prot -out plant_ref_seq;
pigz $biscem/Data/plant_ref_seq.faa;
mv plant_* /home/erwann/Software/ncbi-blast-2.5.0+/blastdb;

Launch actual blastx (vs plant_refseq) :

In [ ]:
### CODE ###
# Launch blastx
biscem='/home/erwann/Desktop/RTDG/BISCEm';
blast='/home/erwann/Software/Blast_2.7.1/bin';
db='/home/erwann/Software/ncbi-blast-2.5.0+/blastdb';

cd $biscem/Output;
for id in 'trinity_cdhitest_renamed_2';
do $blast/blastx -query $id.fasta \
                 -db $db/plant_ref_seq \
                 -out $id'_vs_plant_refseq_prot.tsv' \
                 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart \
                          qend sstart send evalue bitscore qlen slen saccver" \
                 -num_threads 8 \
                 -culling_limit 1;
done;

### 4.4) Parse blastx result

##### sort_plant_hit_vs_plant_refseq_prot.py

In [ ]:
### CODE ###
#!/usr/bin/env python
import os
import sys
import timeit

usage = '\t --------\n' \
        '\t| usage  : python sort_plant_hit_vs_plant_refseq_prot.py f1 f2\n' \
        '\t| input  : f1 = blast.tsv (vs plant_ref_seq)\n' \
        '\t| input  : f2 = seqs.fasta (blastx queries)\n' \
        '\t| indir  : prot.accession2taxid.gz (to map saccver to staxids)\n' \
        '\t| output : "f2"_1.fasta (plant_hit)\n' \
        '\t| output : "f2"_2.fasta (non_plant_hit = a control here, should be empty)\n' \
        '\t| output : "f2"_3.fasta (no_hit)\n' \
        '\t --------'

if len(sys.argv) != 3:
    print(usage)
    sys.exit()

##############
### Step 1 ###
##############
print('\n\tStep 1) Map saccver to staxids')
t0 = timeit.default_timer()

cpt = 0
qseqid_set, saccver_set = set(), set()
with open(sys.argv[1], 'r') as tsv:
    for row in tsv:
        cpt += 1
        columns = row.split('\t')
        qseqid, saccver = columns[0].rstrip(), columns[14].rstrip()
        # Get rid of accession number version
        saccver = saccver.split('.')[0]
        if not qseqid in qseqid_set:
            saccver_set.add(saccver)
        qseqid_set.add(qseqid)

# Use saccver_set as local query against prot.accession2taxid
# create saccver.temp as input for grep
with open('saccver.temp', 'w') as temp:
    for item in saccver_set:
        temp.write(item.rstrip() + "\n")

# map saccver to staxids
cmd = 'zfgrep -f saccver.temp ../Data/all_prot.accession2taxid.gz'
cmd_result = os.popen(cmd).read()
# First attempt
# cmd = ('zgrep -f saccver.temp prot.accession2taxid.gz | awk \'{print $2\"\t\"$3}\' $_')
# grep on unzipped file is faster (3.42 vs 5.28), but file is too big to please me
# cmd = 'fgrep -f saccver.temp all_prot.accession2taxid'

# store grep result in a temp file (easier to parse)
with open('grep.temp', 'w') as temp:
    temp.write(cmd_result)

# Finally, map saccver to staxids in a dic
saccver_to_staxids_dic = {}
with open('grep.temp', 'r') as grep:
    for line in grep:
        field = line.split('\t')
        saccver_to_staxids_dic.setdefault(field[0].rstrip(), field[2].rstrip())

print('\t\t- TSV have ' + str(cpt) + ' lines, containing ' +
      str(len(saccver_set)) + ' unique saccver')
print('\t\t- grep result contains ' + str(len(saccver_set)) + ' lines')
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

##############
### Step 2 ###
##############
print('\tStep 2) Retrieve taxonomic infos for each staxids with efetch')
t0 = timeit.default_timer()

# Use staxids_set as query with efetch & store result
# Don't give more than let's say 500 entries at a time to avoid timeout
staxids_set = set()
for k, v in saccver_to_staxids_dic.items():
    staxids_set.add(v)

staxids_li = list(staxids_set)
staxids_sub_li = [staxids_li[x:x + 500]
                  for x in range(0, len(staxids_li), 500)]
efetch_li = []
for item in staxids_sub_li:
    staxids_input = ','.join(str(z) for z in item)
    # Details about "cmd" : https://www.biostars.org/p/163595/#271497
    cmd = ('efetch -db taxonomy -id ' + staxids_input + ' -format xml | xtract '
           '-pattern Taxon -sep \'@\' -element TaxId,ScientificName -division '
           'LineageEx -group Taxon -if Rank -equals superkingdom -or Rank '
           '-equals kingdom -or Rank -equals phylum -or Rank -equals class'
           ' -or Rank -equals order -or Rank -equals family -or Rank -equals'
           ' genus -sep \'@\' -element Rank,ScientificName')
    cmd_result = os.popen(cmd).read()
    cmd_result_split = cmd_result.split('\n')
    for i in cmd_result_split:
        efetch_li.append(i)
# 257314@Lactobacillus johnsonii NCC
# 533\tsuperkingdom@Bacteria\tphylum@Firmicutes\tclass@Bacilli\torder@Lactobacillales\tfamily@Lactobacillaceae\tgenus@Lactobacillus'

# Create a dict associating key=staxid with value=list=tax_infos
taxonomy_dic = {}
for line in efetch_li:
    field = line.split('\t')
    tax_ids = field[0].split('@')
    # Sometimes more than one staxid is associated to an entry
    # e.g. "170850@3666@Cucurbita hybrid cultivar"
    for i in tax_ids[:-1]:
        taxonomy_dic.setdefault(i, [None, None, None, None, None, None, None])
        for item in field:
            if 'superkingdom@' in item:
                taxonomy_dic[i][0] = item.split('@')[-1]
            elif 'kingdom@' in item:
                taxonomy_dic[i][1] = item.split('@')[-1]
            elif 'phylum@' in item:
                taxonomy_dic[i][2] = item.split('@')[-1]
            elif 'class@' in item:
                taxonomy_dic[i][3] = item.split('@')[-1]
            elif 'order@' in item:
                taxonomy_dic[i][4] = item.split('@')[-1]
            elif 'family@' in item:
                taxonomy_dic[i][5] = item.split('@')[-1]
            elif 'genus@' in item:
                taxonomy_dic[i][6] = item.split('@')[-1]
# '41840': ['Eukaryota', 'Viridiplantae', 'Streptophyta', 'Sphagnopsida', 'Sphagnales', 'Sphagnaceae', 'Sphagnum']

print('\t\t- taxonomy_dic len is ' + str(len(taxonomy_dic)))
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

##############
### Step 3 ###
##############
print('\tStep 3) Assign contigs best hit to plant or non-plant')
t0 = timeit.default_timer()

# Assign contigs best hits to plant or non-plant based on taxonomy_dic infos
qseqid_set, viridi_hit_set, non_viridi_hit_set = set(), set(), set()
with open(sys.argv[1], 'r') as tsv:
    for row in tsv:
        columns = row.split('\t')
        qseqid, saccver = columns[0].rstrip(), columns[14].rstrip()
        # Get rid of accession number version
        saccver = saccver.split('.')[0]
        # Check if we encounter qseqid for the first time <=> best hit
        if not qseqid in qseqid_set:
            if taxonomy_dic[saccver_to_staxids_dic[saccver]][1] == 'Viridiplantae':
                viridi_hit_set.add(qseqid)
            else:
                non_viridi_hit_set.add(qseqid)
        qseqid_set.add(qseqid)

print('\t\t- qseqid_set len is ' + str(len(qseqid_set)))
print('\t\t- viridi_hit_set len is ' + str(len(viridi_hit_set)))
print('\t\t- non_viridi_hit_set len is ' + str(len(non_viridi_hit_set)))
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')

##############
### Step 4 ###
##############
print('\tStep 4) Find contigs with no hits')
t0 = timeit.default_timer()

# Read initial FASTA (f2) & check intersection with viridi_hit_set & non_viridi_hit_set
# We can deduce contigs with no hit from this intersection
cpt = 0
no_hit_set = set()
with open(sys.argv[2], 'r') as fa:
    for line in fa:
        if line.startswith('>'):
            cpt += 1
            line = line.lstrip('>')
            # if line in no_hit_set:
            #    print(line)
            # no_hit_set.add(line)
            fields = line.split()
            no_hit_set.add(fields[0])

before_union = len(no_hit_set)
no_hit_set = no_hit_set - viridi_hit_set
no_hit_set = no_hit_set - non_viridi_hit_set

print('\t\t- number of seqs (>) in FASTA : ' + str(cpt))
print('\t\t- number of headers added to initial set is ' + str(before_union))
print('\t\t- number of res in no_hit_set : ' + str(len(no_hit_set)))
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds\n')


##############
### Step 5 ###
##############
print('\tStep 5) Create output files')
t0 = timeit.default_timer()

# Create input files (sequence IDs list) for seqtk
file_2 = sys.argv[2].split('/')
sample = file_2[-1].split('.')[0]
with open(sample + '_plant_hit.temp', 'w') as out:
    for item in viridi_hit_set:
        out.write(item + "\n")
with open(sample + '_non_plant_hit.temp', 'w') as out:
    for item in non_viridi_hit_set:
        out.write(item + "\n")
with open(sample + '_no_hit.temp', 'w') as out:
    for item in no_hit_set:
        field = item.split()
        # out.write(item)
        out.write(field[0] + '\n')

# Create output files (plant_hit = 1, non-plant_hit = 2, no-hit = 3) with seqtk
os.system('seqtk subseq ' + sys.argv[2] + ' ' + sample +
          '_plant_hit.temp > ' + sample + '_1.fasta')
os.system('seqtk subseq ' + sys.argv[2] + ' ' + sample +
          '_non_plant_hit.temp > ' + sample + '_2.fasta')
os.system('seqtk subseq ' + sys.argv[2] + ' ' + sample +
          '_no_hit.temp > ' + sample + '_3.fasta')

sum_seqs = int(len(viridi_hit_set)) + \
    int(len(non_viridi_hit_set)) + int(len(no_hit_set))
print('\t\t- number of seqs with plant_hit : ' + str(len(viridi_hit_set)))
print('\t\t- number of seqs in non_plant_hit : ' + str(len(non_viridi_hit_set)))
print('\t\t- number of seqs with no_hit : ' + str(len(no_hit_set)))
print('\t\t- sum of the 3 above : ' + str(sum_seqs) +
      ', which should be equal to : ' + str(cpt))
print('\t\t=> ' + str(round(timeit.default_timer() - t0, 3)) + ' seconds')

os.system('rm *.temp')

Here, we need an extra step compared to sort_plant_hit_vs_nt.py : <br>
We need to map saccver to staxids, via the "accession2taxid" file.

Launch the script :

In [4]:
### CODE ###
biscem='/home/erwann/Desktop/RTDG/BISCEm';
cd $biscem/Output;
for id in 'trinity_cdhitest_renamed_2';
do python $biscem/Git/sort_plant_hit_vs_plant_refseq_prot.py $id'_vs_plant_refseq_prot.tsv' $id.fasta;
done;


	Step 1) Map saccver to staxids
		- TSV have 80119 lines, containing 46660 unique saccver
		- grep result contains 46660 lines
		=> 218.973 seconds

	Step 2) Retrieve taxonomic infos for each staxids with efetch
		- taxonomy_dic len is 437
		=> 3.488 seconds

	Step 3) Assign contigs best hit to plant or non-plant
		- qseqid_set len is 55967
		- viridi_hit_set len is 55967
		- non_viridi_hit_set len is 0
		=> 0.255 seconds

	Step 4) Find contigs with no hits
		- number of seqs (>) in FASTA : 92665
		- number of headers added to initial set is 92665
		- number of res in no_hit_set : 36698
		=> 0.171 seconds

	Step 5) Create output files
		- number of seqs with plant_hit : 55967
		- number of seqs in non_plant_hit : 0
		- number of seqs with no_hit : 36698
		- sum of the 3 above : 92665, which should be equal to : 92665
		=> 0.403 seconds
